In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install deepproblog


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cus

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import cv2
from problog.program import PrologString
from problog import get_evaluatable

# Load the MNIST dataset
def load_data(dataset_name='mnist'):
    if dataset_name == 'mnist':
        return mnist.load_data()
    else:
        raise ValueError("Dataset not supported.")

# Preprocess the data
def preprocess_data(x):
    x = x.astype('float32') / 255.0
    x = np.expand_dims(x, -1)
    return x

# Load pre-trained model
def load_trained_model(model_path):
    return load_model(model_path)

# Predict on the test set
def get_predictions(model, x_test):
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    return predicted_classes

# Select correctly classified samples
def select_correct_samples(x_test, y_test, predicted_classes, samples_per_class=100):
    correct_samples = {i: [] for i in range(10)}
    for i, (img, label, pred) in enumerate(zip(x_test, y_test, predicted_classes)):
        if label == pred and len(correct_samples[label]) < samples_per_class:
            correct_samples[label].append(img)
        if all(len(correct_samples[l]) == samples_per_class for l in range(10)):
            break
    for label in correct_samples:
        correct_samples[label] = np.array(correct_samples[label])
        print(f"Number of samples for class {label}: {len(correct_samples[label])}")
    return correct_samples

# Define transformation functions
def apply_rotation(image, angle):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def apply_noise(image, noise_factor=0.5):
    noisy_image = image + noise_factor * np.random.randn(*image.shape)
    noisy_image = np.clip(noisy_image, 0., 1.)
    return noisy_image

def apply_brightness(image, brightness_factor=0.5):
    bright_image = image * (1.0 + brightness_factor)
    bright_image = np.clip(bright_image, 0., 1.)
    return bright_image

# Apply transformations to generate test cases
def generate_test_cases(samples):
    rotations = [apply_rotation(img, angle=25) for img in samples]
    noises = [apply_noise(img, noise_factor=0.2) for img in samples]
    brightnesses = [apply_brightness(img, brightness_factor=0.3) for img in samples]
    return rotations, noises, brightnesses

# Generate test cases for each class
def generate_all_transformations(correct_samples):
    all_transformations = {}
    for label in correct_samples:
        rotations, noises, brightnesses = generate_test_cases(correct_samples[label])
        all_transformations[label] = {
            "rotations": rotations,
            "noises": noises,
            "brightnesses": brightnesses
        }
    return all_transformations

# Test model on transformations
def test_model_on_transformations(model, transformations, label):
    accuracies = {}
    for transform_name, transformed_samples in transformations.items():
        transformed_samples = np.array(transformed_samples)
        transformed_samples = transformed_samples.reshape((-1, 28, 28, 1))
        predictions = model.predict(transformed_samples)
        predicted_classes = np.argmax(predictions, axis=1)
        accuracy = np.mean(predicted_classes == np.array([label] * len(transformed_samples)))
        accuracies[transform_name] = accuracy
    return accuracies

# Compute accuracies for each class and each transformation
def compute_class_accuracies(model, all_transformations):
    class_accuracies = {}
    for label, transformations in all_transformations.items():
        accuracies = test_model_on_transformations(model, transformations, label)
        class_accuracies[label] = accuracies
    return class_accuracies

# Generate Problog code based on the accuracies
def generate_problog_code(class_accuracies):
    problog_code = ""
    for label, acc in class_accuracies.items():
        print(f"Class {label} - Noise Accuracy: {acc['noises']}, Brightness Accuracy: {acc['brightnesses']}, Rotation Accuracy: {acc['rotations']}")
        problog_code += f"    {acc['noises']:.6f}::noise_{label}.\n"
        problog_code += f"    {acc['brightnesses']:.6f}::brightness_{label}.\n"
        problog_code += f"    {acc['rotations']:.6f}::rotation_{label}.\n\n"
        problog_code += f"    correct_noise_{label} :- noise_{label}.\n"
        problog_code += f"    correct_brightness_{label} :- brightness_{label}.\n"
        problog_code += f"    correct_rotation_{label} :- rotation_{label}.\n\n"
        problog_code += f"    wrong_noise_{label} :- \\+correct_noise_{label}.\n"
        problog_code += f"    wrong_brightness_{label} :- \\+correct_brightness_{label}.\n"
        problog_code += f"    wrong_rotation_{label} :- \\+correct_rotation_{label}.\n\n"

    # Generating pair rules for all unique pairs (including self-pairs).     # AND Relation
    for i in range(10):
        for j in range(i, 10):
            problog_code += f"    pair_correct_noise_{i}_{j} :- correct_noise_{i}, correct_noise_{j}.\n"
            problog_code += f"    pair_wrong_noise_{i}_{j} :- wrong_noise_{i}, wrong_noise_{j}.\n"
            problog_code += f"    pair_correct_brightness_{i}_{j} :- correct_brightness_{i}, correct_brightness_{j}.\n"
            problog_code += f"    pair_wrong_brightness_{i}_{j} :- wrong_brightness_{i}, wrong_brightness_{j}.\n"
            problog_code += f"    pair_correct_rotation_{i}_{j} :- correct_rotation_{i}, correct_rotation_{j}.\n"
            problog_code += f"    pair_wrong_rotation_{i}_{j} :- wrong_rotation_{i}, wrong_rotation_{j}.\n\n"

    # Generating global correctness rules for each pair.     # OR Relation
    for i in range(10):
        for j in range(i, 10):
            problog_code += f"    global_correct_noise_{i}_{j} :- pair_correct_noise_{i}_{j}; pair_wrong_noise_{i}_{j}.\n"
            problog_code += f"    global_correct_brightness_{i}_{j} :- pair_correct_brightness_{i}_{j}; pair_wrong_brightness_{i}_{j}.\n"
            problog_code += f"    global_correct_rotation_{i}_{j} :- pair_correct_rotation_{i}_{j}; pair_wrong_rotation_{i}_{j}.\n\n"

    # Generating final global correctness rules
    problog_code += "    final_global_correct_noise :- " + ", ".join([f"global_correct_noise_{i}_{j}" for i in range(10) for j in range(i, 10)]) + ".\n"
    problog_code += "    final_global_correct_brightness :- " + ", ".join([f"global_correct_brightness_{i}_{j}" for i in range(10) for j in range(i, 10)]) + ".\n"
    problog_code += "    final_global_correct_rotation :- " + ", ".join([f"global_correct_rotation_{i}_{j}" for i in range(10) for j in range(i, 10)]) + ".\n"

    # Generating the final overall system global correctness rule
    # AND Relation
    # problog_code += "    final_global_correct_system :- final_global_correct_noise, final_global_correct_brightness, final_global_correct_rotation.\n"
    # OR Relation
    problog_code += "    final_global_correct_system :- final_global_correct_noise; final_global_correct_brightness; final_global_correct_rotation.\n"

    # Generating queries
    for i in range(10):
        problog_code += f"query(correct_noise_{i}).\n"
        problog_code += f"query(correct_brightness_{i}).\n"
        problog_code += f"query(correct_rotation_{i}).\n"
        problog_code += f"query(wrong_noise_{i}).\n"
        problog_code += f"query(wrong_brightness_{i}).\n"
        problog_code += f"query(wrong_rotation_{i}).\n"

    for i in range(10):
        for j in range(i, 10):
            problog_code += f"query(pair_correct_noise_{i}_{j}).\n"
            problog_code += f"query(pair_wrong_noise_{i}_{j}).\n"
            problog_code += f"query(pair_correct_brightness_{i}_{j}).\n"
            problog_code += f"query(pair_wrong_brightness_{i}_{j}).\n"
            problog_code += f"query(pair_correct_rotation_{i}_{j}).\n"
            problog_code += f"query(pair_wrong_rotation_{i}_{j}).\n"
            problog_code += f"query(global_correct_noise_{i}_{j}).\n"
            problog_code += f"query(global_correct_brightness_{i}_{j}).\n"
            problog_code += f"query(global_correct_rotation_{i}_{j}).\n"

    problog_code += "query(final_global_correct_noise).\n"
    problog_code += "query(final_global_correct_brightness).\n"
    problog_code += "query(final_global_correct_rotation).\n"
    problog_code += "query(final_global_correct_system).\n"

    return problog_code

# Function to generate Problog code and evaluate it
def evaluate_problog(class_accuracies):
    problog_code = generate_problog_code(class_accuracies)
    prolog_model = PrologString(problog_code)
    result = get_evaluatable().create_from(prolog_model).evaluate()
    return result

# Main function to run the workflow
def main():
    # Step 1: Load the data
    print("Step 1: Loading the data")
    (x_train, y_train), (x_test, y_test) = load_data()
    print("Data loaded successfully")

    # Step 2: Preprocess the data
    print("Step 2: Preprocessing the data")
    x_test = preprocess_data(x_test)
    print("Data preprocessed successfully")

    # Step 3: Load the pre-trained model
    print("Step 3: Loading the pre-trained model")
    model_path = '/content/drive/MyDrive/ColabNotebooks/mnist_model.h5' # Update this path to your model
    model = load_trained_model(model_path)
    print("Model loaded successfully")

    # Step 4: Get predictions
    print("Step 4: Getting predictions on the test set")
    predicted_classes = get_predictions(model, x_test)
    print("Predictions obtained successfully")

    # Step 5: Select correctly classified samples
    print("Step 5: Selecting correctly classified samples")
    correct_samples = select_correct_samples(x_test, y_test, predicted_classes)
    print("Correctly classified samples selected successfully")

    # Step 6: Generate test cases
    print("Step 6: Generating test cases")
    all_transformations = generate_all_transformations(correct_samples)
    print("Test cases generated successfully")

    # Step 7: Compute accuracies for each class and each transformation
    print("Step 7: Computing accuracies for each class and each transformation")
    class_accuracies = compute_class_accuracies(model, all_transformations)
    print("Accuracies computed successfully")

    # Step 8: Generate Problog code and evaluate
    print("Step 8: Generating Problog code and evaluating")
    result = evaluate_problog(class_accuracies)
    print("Problog evaluation result:")
    for query, value in result.items():
        print(f"{query}: {value}")

if __name__ == "__main__":
    main()


Step 1: Loading the data
Data loaded successfully
Step 2: Preprocessing the data
Data preprocessed successfully
Step 3: Loading the pre-trained model
Model loaded successfully
Step 4: Getting predictions on the test set
313/313 [==============================] - 3s 9ms/step
Predictions obtained successfully
Step 5: Selecting correctly classified samples
Number of samples for class 0: 100
Number of samples for class 1: 100
Number of samples for class 2: 100
Number of samples for class 3: 100
Number of samples for class 4: 100
Number of samples for class 5: 100
Number of samples for class 6: 100
Number of samples for class 7: 100
Number of samples for class 8: 100
Number of samples for class 9: 100
Correctly classified samples selected successfully
Step 6: Generating test cases
Test cases generated successfully
Step 7: Computing accuracies for each class and each transformation
4/4 [==============================] - 0s 5ms/step
Accuracies computed successfully
Step 8: Generating Problog 